# Bayesian Optimisation

In this notebook we carry out bayesian optimisation in order to optimise the dropout rates added to the autoencoder, and hence reduce the overfitting previously observed during training.

This notebook should be adapted accordingly for use with the STM or MNIST datasets.

---


In [ ]:
import numpy as np

import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from keras import models

from keras.models import Model
from keras.layers import Input, Dense, Flatten, Reshape, Conv2D, Conv2DTranspose , Dropout
from keras.optimizers import Adam
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.preprocessing import image

from hyperopt import fmin , tpe , hp , STATUS_OK , Trials

import os
import cv2

---

## Step1: Importing the STM dataset.

---

In [ ]:
### importing the STM Images ###

# loading folder
path = "..."
image_folder = [f for f in os.listdir( ... ) if f.endswith((".jpg", ".png", ".jpeg"))]

# ----------- extracting images from folder ----------- #

all_images = []

for i, image_file in enumerate( image_folder ):
    
    image_path = os.path.join( path, image_file )
    img        = cv2.imread( image_path )
    
    all_images.append( img )

# normalising images
all_images = np.array( all_images )
all_images = all_images / np.max(all_images)

In [ ]:
### Defining Datasets ##

kTraining_images_stm   = all_images[:600]      # 600
kValidation_images_stm = all_images[600:-1]    # 150
kTesting_images_stm    = all_images[-1:]       #   1

---

In [ ]:
### importing the MNIST Dataset ###

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalisation
train_images = train_images / np.max(train_images)
test_images  = test_images / np.max(test_images)

# Dividing images
kTraining_images_mnist   = train_images            # 60,000
kValidation_images_mnist = test_images[:5000]      #  5,000
kTesting_images_mnist    = test_images[5000:]      #  5,000


---

## Step 2: Re-shaping images.

---

In [ ]:
### Re-shaping Images ###

def resize_and_convert( images, shape ):
    
    images_expanded = tf.expand_dims( images , -1)
    
    rgb_images = tf.image.grayscale_to_rgb( images_expanded )
    
    resized_images = tf.image.resize( rgb_images , shape)
    
    return resized_images

# ---------------------------------------------------------- #

new_shape = [64, 64]

kTraining_images   = resize_and_convert( kTraining_images_mnist,   new_shape )
kValidation_images = resize_and_convert( kValidation_images_mnist, new_shape )
kTesting_images    = resize_and_convert( kTesting_images_mnist,    new_shape )


---

## Step 3: Defining model.

---

In [ ]:

def creating_model( params ):
    '''
    Creating and compiling an Autoencoder Model.
    
    INPUTS:
    - params => parameters to optimize (dropout in this case).
    OUTPUTS:
    - autoencoder_model => Autoencoder Model Compiled.
    '''

    # -------------------------- Encoder -------------------------- #
    
    input_img_y = Input(shape=(64, 64, 3))

    y = Conv2D( 32 , (3, 3), activation='relu', padding='same', strides=2)(input_img_y)
    y = Dropout( params['dropout_1'] )(y)
    y = Conv2D( 64 , (3, 3), activation='relu', padding='same', strides=2)(y)
    y = Dropout( params['dropout_2'] )(y)
    y = Conv2D( 128 , (3, 3), activation='relu', padding='same', strides=2)(y)
    y = Dropout( params['dropout_3'] )(y)
    y = Conv2D( 256 , (3, 3), activation='relu', padding='same', strides=2)(y)
    y = Dropout( params['dropout_4'] )(y)

    # -------------------------- Latent Layers -------------------- #
    shape_before_flattening = (4, 4, 256)
    
    encoder_y = Flatten()(y)
    y = Dense( 32 , activation='relu')(encoder_y)
    y = Dropout( params['dropout_5'] )(y)
    y = Dense(np.prod(shape_before_flattening))(y)
    y = Dropout( params['dropout_6'] )(y)
    
    # -------------------------- Decoder -------------------------- #

    y = Reshape(shape_before_flattening)(y)
    y = Conv2DTranspose( 256 , (3, 3), activation='relu', padding='same', strides=2)(y)
    y = Dropout( params['dropout_7'] )(y)
    y = Conv2DTranspose( 128 , (3, 3), activation='relu', padding='same', strides=2)(y)
    y = Dropout( params['dropout_8'] )(y)
    y = Conv2DTranspose( 64 , (3, 3), activation='relu', padding='same', strides=2)(y)
    y = Dropout( params['dropout_9'] )(y)
    y = Conv2DTranspose( 32 , (3, 3), activation='relu', padding='same', strides=2)(y)
    y = Dropout( params['dropout_10'] )(y)

    output_y = Conv2DTranspose(3, (3, 3), activation='sigmoid', padding='same')(y)

    # ----------- Autoencoder Model ----------- #
    
    autoencoder_model = Model( input_img_y, output_y )
    autoencoder_model.compile(optimizer="adam",
                              loss="mean_squared_error",
                              metrics=['mean_squared_error','accuracy'])
    
    return autoencoder_model

# ============================================================================== #

def objective( params ):
    '''
    Function which trains a model created by the "creating model" function...
    ... in order to find the MSE certain parameters result in.
    INPUTS:
    - params => parameters for which we want to find MSE.
    OUTPUTS:
    - {'loss': val_MSE, 'status': STATUS_OK} => returns loss and status of model training.
    '''
    
    # Getting model
    model = creating_model( params )
    
    # Training model
    history = model.fit( kTraining_images, kTraining_images,
                        epochs = 150,
                        validation_data = ( kValidation_images, kValidation_images ),
                        batch_size = 32,
                        verbose = 0)
    
    # Getting the mean squared error
    val_MSE = np.amin( history.history["val_mean_squared_error"] )
    
    return {'loss': val_MSE, 'status': STATUS_OK}


---

## Step 4: Bayesian optimisation.

---

In [ ]:
### Hyperparametrisation Optimization ###

# parameter ranges we're going to test
_space_ = {
    'dropout_1': hp.uniform('dropout_1', 0.0, 0.5),
    'dropout_2': hp.uniform('dropout_2', 0.0, 0.5),
    'dropout_3': hp.uniform('dropout_3', 0.0, 0.5),
    'dropout_4': hp.uniform('dropout_4', 0.0, 0.5),
    'dropout_5': hp.uniform('dropout_5', 0.0, 0.5),
    'dropout_6': hp.uniform('dropout_6', 0.0, 0.5),
    'dropout_7': hp.uniform('dropout_7', 0.0, 0.5),
    'dropout_8': hp.uniform('dropout_8', 0.0, 0.5),
    'dropout_9': hp.uniform('dropout_9', 0.0, 0.5),
    'dropout_10': hp.uniform('dropout_10', 0.0, 0.5) }

# Running the optimization using Hyperopt
_trials_ = Trials()

# Carries out the optimization
best = fmin( fn = objective,
            space = _space_,
            algo = tpe.suggest,
            max_evals = 50,
            trials = _trials_)


---